In [1]:
import pandas as pd
import numpy as np
import re, os, pickle
from os.path import join
from tqdm import tqdm
from datetime import datetime

### 시효완성일 > 시효완성월 작업하기

In [4]:
df = pd.read_excel(r"D:\3.자산\전산 dataset\솔림\230926\output2\POOL8_데이터_230926_대주발송용.xlsx", sheet_name="무담보", dtype=str)

In [7]:
# 마지막 작업 : 시효완성월 전체 수정(새시효완성일에 대해)
basedate = "230926"
기준일 = pd.Timestamp("20"+basedate[:4]+"01")

# 함수벡터화
df["시효완성일"] = pd.to_datetime(df["시효완성일"])  # 시효완성일을 날짜 형식으로 변환
df["시효완성월_계산"] = (df["시효완성일"].dt.year - 기준일.year) * 12 + (df["시효완성일"].dt.month - 기준일.month)

# 버릴 시효완성월_ 작성
x = df["시효완성월_계산"]
conds = [(x<0) | pd.isna(x), x<1, x<2, x<3, x<4, x<5, x<6, x<12, x>=12]
values = ["시효만료", "당월도래", "1개월", "2개월", "3개월", "4개월", "5개월", "1년이내", "1년이후"]
df["시효완성월_계산"] = np.select(conds, values)

# 시효완성월과 시효사유 : 압류로 인한 중단 반영
df['시효완성월'] = np.where(df.시효완성월=="중단", "중단", df.시효완성월_계산)

# df 저장 : 타입원복 및 불필요한 열 제거
df = df.iloc[:,:-1] # 시효완성월_계산 제거
df["시효완성일"] = df["시효완성일"].apply(lambda x: pd.Timestamp(x).strftime("%Y-%m-%d") if pd.notna(x) else "")

df.to_excel(r"D:\3.자산\전산 dataset\솔림\230926\output2\xx.xlsx", index=False)